# Mission to Mars
## (Simulating and Controlling the $N$-body Problem)

<br> 

### 6 April 2020

<br>

### James Kermode (School of Engineering), 
### Another Author (Mathematics Institute), 
### ...
### Final Author (Department of Economics)

An essay submitted for assessment of the group project for the module `IL027 Computer Modelling`.

In [ ]:
include("aux.jl")

## Introduction 

**[[The Introduction should provide some background, motivate the topic/problem of the essay and provide a high level summary.]]**

The $N$-body problem is a classical system of ordinary differential equation that does not possess a general closed from solution. **[[add some historical context, e.g., Newton, what solution methods are there? add some references]]** The first goal of this essay is to discuss how the $N$-body problem can be solved on a computer, using modern numerical solvers for ordinary differential equations. 

In practise, one often encounters a scenario where a system of ODEs has certain input parameters, called *controls*, which allow us to adjust the solution trajectory in order to achieve a specific aim. For example, consider the task of landing space craft on a moon or a planet. The engine can orient as well as slow the descent. The second aim of this essay is to describe how to solve this classical *control problem*. 

Finally **[[brief description of some creative application or twist you want  to pursue]]** 

For the purposes of presentation we will present the problems described above in the context of an Alian attack: imagine the year is 2044 and human colonies are already established on Mars. An alien race has entered the solar system planning to attack and conquer Mars and the Earth.

<img src="img/Alien_attack.jpg" alt="Alien_attack" style="width: 400px;" align="middle" />

**[[NOTE: we do not require a "story" in this essay, it can be good fun and effective, but only if it fits the technical questions/challenges. The focus MUST be on the latter!]]**

## 1. Escape velocity of a ballistic missile

The UK Space Agency (UKSA) plans to launch a spacecraft to defend the Earth from aliens. 

<img src="img/UKSA.jpg" alt="UKSA" style="width: 300px;" align="middle"/>

Engineers need to calculate the minimum velocity (called escape velocity $v_e$) with which the missile must be launched in order to escape the Earth. Assuming the Earth is a spherical symmetric massive body with no atmosphere, the escape velocity $v_e$ of any object launched on Earth’s surface is:

$$ v_e = \sqrt{\frac{2GM_e}{R_e}} $$

where $G$ is the universal gravitational constant, $M_e$ the mass of the Earth, and $R_e$ is the radius of the Earth. Note that this equation is independent of the mass of the object. The following sketch illustrates the idea of launching an object to space at different velocities:

<img src="img/Escape_velocity.jpg" alt="Escape_velocity" style="width: 300px;" align="middle"/>

We can therefore compute the escape velocity as follows: 

In [ ]:
G = 6.67408e-11  # m^3 kg^{-1} s^{-2}
Mₑ = 5.972e24    # kg
Rₑ = 6.356e6     # m
# unit for vₑ : sqrt( m^3 kg^{-1} s^{-2} * kg / m) = sqrt(m^2/s^2) = m/s ✓
vₑ = sqrt(2 * G * Mₑ / Rₑ)   
println("Escape Velocity = $(round(Int,vₑ)) m/s")

## 2. Models and Methods

As a response strategy, the UKSA decided to send a spacecraft to Mars to defend the human colonies there. To do this, it is necessary to know the trajectories of Earth and Mars with respect to the Solar System frame of reference. Trajectories of planets and stars are inter-dependent, since they mutually attract each other due to gravity. This makes the process of prediction a non-trivial task.

### 2.1 The 3-body problem

A group of student interns at UKSA decide to model the trajectories of Earth and Mars using Newtonian gravity. They make the following assumptions: 

- the planets are moving in a two-dimensional plane, 
- the effect of the rest of the planets of the Solar System on the trajectories of the Earth and Mars is negligible. 

Any object subjected to a gravitational field has potential energy $U$:

$$ U = -G \frac{m_1 m_2}{r_{12}} $$

where $G$ is the universal gravitational constant, $m_1$ the mass of the object creating a gravitational field, $m_2$ the mass of the object subjected to the gravitational field and $r_{12}$ is the distance between the objects. In this case, the Earth is subjected to the gravitational fields of the Sun and Mars, and vice versa. The total potential energy with $N$ bodies is

$$ U(\mathbf{R}) = -\frac{G}{2} \sum_{i=1}^{N} \sum_{j=1\\i \ne j}^{N} \frac{m_{i} m_{j}}{r_{ij}} $$

where the sum runs over the $N$ bodies, avoiding the self-interaction case $i=j$ and the factor of $\frac{1}{2}$ is to avoid double counting and the vector $\mathbf{R}$ collects the positions of all bodies - for our example with the Sun, Earth and Mars, $\mathbf{R} = (\mathbf{r}_s, \mathbf{r}_e, \mathbf{r}_m)$. Throughout, we will use the convention that vectors denoted by a capital letter (e.g. $\mathbf{R}$) are for the full system, while those denoted by a lower case letter are for an individual body (e.g. $\mathbf{r}_i$).

This is an example of a [3-body problem](https://en.wikipedia.org/wiki/Three-body_problem), or more generally, an $N$-body problem.

In [ ]:
D = 2 # dimensionality of space (2D, 3D etc.)
N = 3 # number of bodies - currently Sun, Earth, Mars - we will increase this later

function U(R::AbstractVector, m::AbstractVector)
    r = unpack_R(R) # convert from 1d vector back to D-dimensional (here 2D)
    U = 0.0
    for i=1:N-1
        for j=i+1:N
            r_ij = norm(r[i, :] - r[j, :]) # distance from i to j
            U += -G * m[i] * m[j] / r_ij # contribution to potential
        end
    end
    return U
end

pack_R(R::AbstractMatrix) = R[:]
unpack_R(R::AbstractVector) = reshape(R, (N, D))

### Masses of planetary bodies

For calculating the trajectories of Mars and Earth, we need to know the masses of the Sun, Earth and Mars, as well as their initial positions and velocities. 

The masses of these three bodies are respectively: $1.989 \times 10^{30}$ kg, $5.972 \times 10^{24}$ kg, and $6.39 \times 10^{23}$ kg. Since these numbers are huge, we scale them by using solar mass units $M_s$, where $M_s = 1.989 \times 10^{30}$ kg. Then, the masses are respectively: $1 M_s$, $0.000003 M_s$, and $0.00000032 M_s$:

In [ ]:
# Masses in solar mass units
m_s = 1.0  
m_e = 0.000003 
m_m = 0.00000032
m = [m_s, m_e, m_m] # pack masses into a 1D array
# M = repeat(m, D);   # repeat masses for each dimension

### Distances in astronomical units

Similarily, we scale the distance by using Astronomical Units (A.U.), where 1 A.U. = $1.5 × 10^{11}$ m (which is about the average distance from the Earth to the Sun), and defined the sun to be at coordinates (0,0) in the 2-dimentional plane. The spacecraft is going to be launched on April 1st, 2044, for which the UKSA estimates that the positions of the Sun, Earth and Mars are:

In [ ]:
# Initial position vectors in X,Y coordinates
r_s = [0.0, 0.0]
r_e = [0.1, -1.0]   # In astronomical units
r_m = [0.0, -1.524] # In astronomical units
R0 = pack_R([r_s r_e r_m]');  

**[[ NOTE: Here, the lecture had a description of how to pack and unpack vectors. This has no place in the essay. Focus on the description of the task, and the modelling. The code speaks for itself. If in doubt add a brief comment. However, you may wish to comment on certain choices of algorithms or packages that you will use.]]**

### Gravitational forces

Newton's second law relates the forces felt by each body $i = 1 \ldots N$ to their acceleration:

$$\mathbf{f}_i = m_i \mathbf{a}_i$$

$$-\nabla_i U(\mathbf{R}) = m_i \frac{d \mathbf{v}_i}{dt} = m \frac{d^2\mathbf{r}_i}{dt^2}$$

By taking the derivative of the potential energy with respect to the positions, the force of attraction between the bodies can be calculated via 

$$\mathbf{F} = -\nabla U(\mathbf{R}) = \left(-\frac{\partial U}{\partial x_1}, -\frac{\partial U}{\partial x_2}, \ldots \right) $$. 

In [ ]:
∇U(R) = ForwardDiff.gradient(R -> U(R, m), R);

### Setting the initial velocities

The time units used are years, so the velocity is measured in A.U./year. The initial velocities of the three bodies for the year 2044 are:

In [ ]:
# Initial velocity vectors in X,Y coordinates
v_s = [0.0, 0.0] # In astronomical units per year
v_e = [-6.32, 0.0] # In astronomical units per year
v_m = [-5.05, 0.0] # In astronomical units per year
V0 = pack_R([v_s v_e v_m]');

The change in units leads the gravitational constant to change as well. Its new value is:

In [ ]:
G = 37.95  # Gravitational constant

The travel time for the spacecraft to go from Earth to Mars was calculated to be around 1 year. The aliens were expected to arrive in May 2045, so the spacecraft would arrive just in time to defend the human colonies in Mars. 

We will first run our simulation for 2 years to check it's behaviour.

In [ ]:
# Simulation time variables
dt = 0.005
t = 0.0:dt:2
T = length(t);

## 2.2 Numerical Integrators 

### Forward Euler integration

If we use the Forward Euler method to integrate the equation of motion we obtain

$$ \mathbf{v}_i(t + \Delta t) = \mathbf{v}_i(t) + \mathbf{a}_i(t) \Delta t $$
$$ \mathbf{r}_i(t + \Delta t) = \mathbf{r}_i(t) + \mathbf{v}_i(t) \Delta t $$

where $\mathbf{r}_i$,  $\mathbf{r}_i$ and $\mathbf{r}_i$ denote the position, velocity and acceleration of body $i$, respectively.  The accelerations are given by $\mathbf{a}_i = -\nabla_i U(\mathbf{R}) / m_i$ from Newton's second law.

We can combine the integration for all the particles using vectors $\mathbf{R}$, $\mathbf{V}$ and $\mathbf{A}$

$$ \mathbf{V}(t + \Delta t) = \mathbf{V}(t) + \mathbf{A}(t) \Delta t $$
$$ \mathbf{R}(t + \Delta t) = \mathbf{R}(t) + \mathbf{V}(t) \Delta t $$


We'll write a function `forward_euler()` to do implement this, and test it on our system. Note how similar the code is to the equations above!

In [ ]:
function forward_euler(∇U, R0, V0, m, dt, T)
    n = length(R0)
    R = copy(R0)        # current position
    V = copy(V0)        # current velocity
    Rs = zeros(T, n)    # position history (trajectory) 
    Vs = zeros(T, n)    # velocity history
    Hs = zeros(T)       # total energy history 
    M = repeat(m, D)

    for i = 1:T
        Rs[i, :] = R    # Store positions
        Vs[i, :] = V    # Store velocities
        Hs[i]    = U(R, m) + 0.5 * dot(M.*V, V)
        
        V_i = copy(V)   # Store current values of velocities
        A = -∇U(R) ./ M # acceleration from Newton's 2nd law (F = ma)
        V += A * dt     # Update velocities
        R += V_i * dt   # Update positions
    end
    return Rs, Vs, Hs
end

R1, V1, H1 = forward_euler(∇U, R0, V0, m, dt, T);

We can now plot the trajectories obtained with the Sun in yellow, Earth in blue, Mars in red. We observe that there is drift over time, with the Earth not returning to it's old trajectory in the second year! We either need to reduce the timestep $\Delta t$ or find a more accurate integration scheme.

In [ ]:
plot_trajectory(R1)

### Symplectic Euler

The [semi-implicit Euler method](https://en.wikipedia.org/wiki/Semi-implicit_Euler_method) or symplectic Euler method is a very small modification of the Euler method which conserves energy and has much improved performance. This method is obtained by simply updating the velocity before using it to update the position:

$$ \mathbf{v}_i(t + \Delta t) = \mathbf{v}_i(t) + \mathbf{a}_i(t) \Delta t $$
$$ \mathbf{r}_i(t + \Delta t) = \mathbf{r}_i(t) + \mathbf{v}_i(t + \Delta t) \Delta t $$


In [ ]:
function symplectic_euler(∇U, R0, V0, m, dt, T)
    n = length(R0)
    R, V = copy(R0), copy(V0)
    Rs, Vs, Hs = zeros(T, n), zeros(T, n),  zeros(T)
    M = repeat(m, D)

    for i = 1:T
        Rs[i, :] = R     # Store positions
        Vs[i, :] = V     # Store velocities
        Hs[i]    = U(R, m) + 0.5 * dot(M.*V, V)
        
        A = -∇U(R) ./ M  # acceleration from Newton's 2nd law (F = ma)
        V += A * dt      # Update velocities
        R += V * dt      # Update positions
    end
    return Rs, Vs, Hs
end

R2, V2, H2 = symplectic_euler(∇U, R0, V0, m, dt, T);

Plotting the solution, we observe that trajectories are now approximately periodic. This is a signature of energy conservation.

In [ ]:
plot_trajectory(R2)

We can make our claim about energy conservation more quantitative, by integrating over a longer time period and then plotting the total energies for the Forward Euler and the Symplectic Euler schemes. We observe, however, that total energy is not exactly conserved but only approximately. We give a brief explanation of this effect in Appendix A.

In [ ]:
t = 0.0:dt:20
T = length(t)
_, _, H1 = forward_euler(∇U, R0, V0, m, dt, T)
_, _, H2 = symplectic_euler(∇U, R0, V0, m, dt, T)
plot(; ylims = [-7e-5, -1e-5], ylabel = "total energy", xlabel = "time", legend = :topleft)
plot!(t, H1, lw=2, label = "Forward Euler")
plot!(t, H2, lw=2, label = "Symplectic Euler")

## 3. Results

**[[ describe and carry out the actual simulation of interest, referencing as needed the methodology introduced in Sec. 2.1, 2.2. Discuss the results. In this particular case, you would setup realistic trajectories of the Sun, Earth and Mars, possibly the entire solar system and then determine how to launch a space-craft from earth to reach Mars orbit. (this might require another Methodology secion on control theory first)
]]**

## 4. Conclusions

**[[In this section give a brief summary of what you achieve in this essay and of the conclusions of your investigation. This could be about 1/3 to 1 page long]]**

In the first part of this essay we investigated the use of different discretisation methods to numerically solve the $N$-body problem. We demonstrated that the Forward Euler scheme exhibits significant energy drift, which severely limits its utility for qualitatively accurate long-time simulations. By contrast, the symplective Euler method conserves the total energy of the system, up to some controllable approximation. 

We then applied the symplectic Euler method to .... **[[add details]]**  In summary, we have presented a methodology to reliably control the launching of spaceships from Earth's surface into Mars' orbit. 

Future work will need to address ... **[[what are the limitations of your work, what are the assumptions you've made, etc]]**

###  Appendix A Proof of Energy Conservation

**[[This section is an example of an elementary but interesting mathematical result one could add]]** 

We consider a quadratic hamiltonian, 
$$
    H(q, p) = \frac12 |p|^2 +  \frac12 q^T A q,
$$
where $q$ is an abstract position variable, $p$ a momentum, $\frac12 |p|^2$ is the kinetic energy (all masses are $1$) and $\frac12 q^T A q$ models potential energy. The associated Hamiltonian dynamics is 
$$ \begin{align*}
    \dot{q} &= p, \\ 
    \dot{p} &= - A q
\end{align*}$$
One can readily check that this dynamical system conserves total energy, i.e., 
$$
    \frac{d}{dt} H(q(t), p(t)) = 0.
$$
Similarly, one can establish that a forward Euler discretisation $(q_n, p_n)$ would satisfy $H(p_n, q_n) \to \infty$ as $n \to \infty$. By contrast, the symplectic Euler method, approximately conserves energy. This is formalised in the following theorem.

**Theorem:** Consider the symplectic Euler method for a quadratic Hamiltonian, 
$$ \begin{align*}
    q_{n+1} &= q_n + h p_n, \\ 
    p_{n+1} &= p_n - h A q_{n+1},
\end{align*}$$
where $h > 0$ is the time step. Then, 
$$
    H_h(q_n, p_n) = {\rm const}
$$
where $H_h$ is a "shadow hamiltonian" given by 
$$
    H_h(q, p) = H(q, p) + \frac12 h p^T A q.
$$

**Proof:** This can be checked with a direct algebraic calculation.


For further details on the mathematics theory of symplectic numerical integration, see (Hairer, Lubich Wanner; Geometric Numerical Integration; Springer 2006; https://doi.org/10.1007/3-540-30666-8)


**FURTHER COMMENTS:**

1. I did not edit the rest of this notebook to convert it into an "essay style". Hopefully this is sufficient so far to give you an idea what we expect. Any questions, please post on Moodle.

2. The Background, Methods, Results outline I followed here is quite standard in the sciences, and is something we don't require but which we do recommend. Variations are possible of course. E.g., if I were to follow L4, then I would probably use (1) Background, (2) N-Body Problem [(2.1) Model, (2.2) Numerical Discretisation, (2.3) Results], (3) Mars Lander [(3.1) Control problems, (3.2) numerical optimisation, (3.3) Results].

3. Appendix A  is an example of going a little deeper into a specific topic of interest, in this case a mathematical explanation of a numerical observation. I chose this because it seemed to fit this particular essay. But we emphasize that there is no requirement or expectation to add technical sections such as this one! Other "further topics" that might be equally appropriate include:
    * a brief explanation of a mathematical theory (avoid detailed proofs)
    * background on a specific key  algorithm you are using, e.g. brief explanation, history, references
    * description of statistical analysis
    * historical context
    * additional / more detailed data that might overwhelm the main article but is still interesting to include for reference
    * ... (be creative, and make use or your unique expertise!) ...

4. Please, please, please make sure the notebook generates all output correctly if restarted from [Kernel > Restart &  Run All].

5. it is generally good practise to cite peer-reviewed sources rather than Wikipedia, blogs, etc.